In [1]:
import numpy as np
import pandas as pd
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
def tokenizer(text):
    '''
    Возвращает для каждого слова все строки, вне зависимости от скора.\
    Пропускает знаки препинания и числа во входном тексте.\
    !!!На вход принимает строку!!!\
    Также создаёт столбец 'POS' с частью речи, теги прописывает на двух языках.
    '''
    morph = pymorphy2.MorphAnalyzer()
    anal = pd.DataFrame()
    text_no_trash = ''.join(ch for ch in text if (ch not in set(string.punctuation))&(ch not in '0123456789'))
    for (i,word) in zip(range(len(text_no_trash.split())),text_no_trash.split()):
        word_anal = pd.DataFrame(morph.parse(word))
        word_anal['number'] = i+1
        anal = pd.concat([anal,word_anal])
    anal.reset_index(drop=True,inplace=True)
    anal['tag_cyr'] = ''
    for i in list(anal.index):
        anal.loc[i,'tag_cyr'] = morph.lat2cyr(anal.loc[i,'tag'])
        anal.loc[i,'POS'] = str(anal.loc[i,'tag'])[:4]
    return(anal)

In [3]:
data = pd.read_csv('morph_check.csv',sep=',',encoding='cp1251')

In [4]:
data.head()

,word,normal_form,POS,tag
0,цветки,цветок,NOUN,"NOUN,inan,masc plur,nomn"
1,дикорастущих,дикорастущий,ADJF,"ADJF plur,gent"
2,форм,форма,NOUN,"NOUN,inan,femn plur,gent"
3,обыкновенной,обыкновенный,ADJF,"ADJF,Qual femn,sing,gent"
4,сирени,сирень,NOUN,"NOUN,inan,femn sing,gent"


In [5]:
#нумеруем слова по порядку, чтобы потом правильно джойнить таблицы
for i in data.index:
    data.loc[i,'number'] = i+1

data.head()

,word,normal_form,POS,tag,number
0,цветки,цветок,NOUN,"NOUN,inan,masc plur,nomn",1.0
1,дикорастущих,дикорастущий,ADJF,"ADJF plur,gent",2.0
2,форм,форма,NOUN,"NOUN,inan,femn plur,gent",3.0
3,обыкновенной,обыкновенный,ADJF,"ADJF,Qual femn,sing,gent",4.0
4,сирени,сирень,NOUN,"NOUN,inan,femn sing,gent",5.0


In [6]:
#готовим текст для анализа; помним, что наш токенайзер кушает на входе строку
text = ''.join('{} '.format(ch) for ch in data.word)

text

'цветки дикорастущих форм обыкновенной сирени мелкие парными пучками по цветков в пучке отдельные сорта по цветков в конце осени кора побегов становится серой в центре цветка белокремовое пятнышко в виде звёздочки при уплотнении почвы и асфальтировании суховершинит морозостоек родина горные леса тяньшаня дико растёт на сухих склонах в умеренно тёплых районах на обширной территории от западной европы до восточной азии '

In [7]:
result = tokenizer(text)

result

,word,tag,normal_form,score,methods_stack,number,tag_cyr,POS
0,цветки,"NOUN,inan,masc plur,nomn",цветок,0.500000,"((<DictionaryAnalyzer>, цветки, 3007, 6),)",1,"СУЩ,неод,мр мн,им",NOUN
1,цветки,"NOUN,inan,masc plur,accs",цветок,0.500000,"((<DictionaryAnalyzer>, цветки, 3007, 12),)",1,"СУЩ,неод,мр мн,вн",NOUN
2,дикорастущих,"ADJF plur,gent",дикорастущий,0.333333,"((<DictionaryAnalyzer>, дикорастущих, 162, 21),)",2,"ПРИЛ мн,рд",ADJF
3,дикорастущих,"ADJF anim,plur,accs",дикорастущий,0.333333,"((<DictionaryAnalyzer>, дикорастущих, 162, 23),)",2,"ПРИЛ од,мн,вн",ADJF
4,дикорастущих,"ADJF plur,loct",дикорастущий,0.333333,"((<DictionaryAnalyzer>, дикорастущих, 162, 26),)",2,"ПРИЛ мн,пр",ADJF
5,форм,"NOUN,inan,femn plur,gent",форма,1.000000,"((<DictionaryAnalyzer>, форм, 55, 8),)",3,"СУЩ,неод,жр мн,рд",NOUN
6,обыкновенной,"ADJF,Qual femn,sing,gent",обыкновенный,0.250000,"((<DictionaryAnalyzer>, обыкновенной, 2189, 8),)",4,"ПРИЛ,кач жр,ед,рд",ADJF
7,обыкновенной,"ADJF,Qual femn,sing,datv",обыкновенный,0.250000,"((<DictionaryAnalyzer>, обыкновенной, 2189, 9),)",4,"ПРИЛ,кач жр,ед,дт",ADJF
8,обыкновенной,"ADJF,Qual femn,sing,ablt",обыкновенный,0.250000,"((<DictionaryAnalyzer>, обыкновенной, 2189, 11),)",4,"ПРИЛ,кач жр,ед,тв",ADJF
9,обыкновенной,"ADJF,Qual femn,sing,loct",обыкновенный,0.250000,"((<DictionaryAnalyzer>, обыкновенной, 2189, 13),)",4,"ПРИЛ,кач жр,ед,пр",ADJF


In [2]:
#присоединяем к анализу правильные результаты

check = pd.merge(result,data,how='left',on='number',suffixes=('_pym','_real'))

result.shape,data.shape,check.shape

NameError: name 'pd' is not defined

In [9]:
#создаём флаг, если результат анализа pymorpy2 совпал с реальным значением
check.loc[check.POS_pym==check.POS_real,'POS_right'] = 1
check.loc[check.normal_form_pym==check.normal_form_real,'NF_right'] = 1
for i in list(check.index):
    check.loc[i,'tag_right'] = (str(check.loc[i,'tag_pym'])==str(check.loc[i,'tag_real']))*1

#взвешиваем на скор
check['POS_right_w'] = check.score*check.POS_right
check['NF_right_w'] = check.score*check.NF_right
check['tag_right_w'] = check.score*check.tag_right

In [1]:
#группируем по каждому слову и находим accuracy

check.groupby('number',as_index=False).sum().mean()

NameError: name 'check' is not defined